# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/Weatherpy.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bengkulu,-3.8004,102.2655,85.55,64,22,9.46,ID,2021-02-04
1,1,arraial do cabo,-22.9661,-42.0278,75.88,89,0,24.52,BR,2021-02-04
2,2,butaritari,3.0707,172.7902,81.59,84,78,16.42,KI,2021-02-04
3,3,barrow,71.2906,-156.7887,-27.40,68,1,13.80,US,2021-02-04
4,4,tuktoyaktuk,69.4541,-133.0374,-40.00,67,1,6.91,CA,2021-02-04


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store Lat and Lng into Locations
locations = cities_df[["Lat","Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

# Create a humidity heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [11]:
ideal_conditions_df = cities_df.loc[(cities_df["Max temp"] > 20) & (cities_df["Max temp"] < 30) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] <50)]
ideal_conditions_df = ideal_conditions_df.dropna(how="any")
ideal_conditions_df.head()

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,hamilton,39.1834,-84.5333,26.01,93,1,1.72,US,2021-02-04
118,118,kodiak,57.7900,-152.4072,26.60,86,1,3.44,US,2021-02-04
216,216,waynesboro,38.0685,-78.8895,28.40,46,1,3.44,US,2021-02-04
245,245,hashtrud,37.4779,47.0508,28.53,88,2,1.74,IR,2021-02-04
272,272,bend,44.0582,-121.3153,28.40,80,1,4.61,US,2021-02-04


### Hotel Map

In [12]:
# Set up new hotel dataframe
hotel_df = ideal_conditions_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
64,hamilton,US,39.1834,-84.5333,
118,kodiak,US,57.7900,-152.4072,
216,waynesboro,US,38.0685,-78.8895,
245,hashtrud,IR,37.4779,47.0508,
272,bend,US,44.0582,-121.3153,
306,garden city,US,42.3256,-83.3310,
315,collingwood,CA,44.4834,-80.2164,
421,southbridge,US,42.0751,-72.0334,


In [13]:
# set up parameters dictionary
params = {"type": "hotel", "keyword": "hotel", "radius": 5000, "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up loop through ideal_conditions data frame
for index, row in hotel_df.iterrows():
    
    # retrieve city, lat and lng from df
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add keyword to parameters dictionary
    params["location"] = f"{lat},{lng}"
    
    # create url and call API request
    response = requests.get(base_url,params=params).json()
    
    results = response["results"]
    
    # Save the hotel name to dataframe
    try:
        print(f"There is a hotel in {city} named {results[0]['name']}")
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
        
    except(KeyError,IndexError):
        print("missing field, skip")
        
print("----End of Search----")

There is a hotel in hamilton named Graduate Cincinnati
There is a hotel in kodiak named Best Western Kodiak Inn And Convention Center
There is a hotel in waynesboro named The Iris Inn
There is a hotel in hashtrud named هتل ماهان هشترود
There is a hotel in bend named Riverhouse on the Deschutes
There is a hotel in garden city named Country Inn & Suites by Radisson, Dearborn, MI
There is a hotel in collingwood named Georgian Bay Hotel, Trademark Collection by Wyndham
There is a hotel in southbridge named Comfort Inn & Suites
----End of Search----


In [14]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
64,hamilton,US,39.1834,-84.5333,Graduate Cincinnati
118,kodiak,US,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
216,waynesboro,US,38.0685,-78.8895,The Iris Inn
245,hashtrud,IR,37.4779,47.0508,هتل ماهان هشترود
272,bend,US,44.0582,-121.3153,Riverhouse on the Deschutes
306,garden city,US,42.3256,-83.3310,"Country Inn & Suites by Radisson, Dearborn, MI"
315,collingwood,CA,44.4834,-80.2164,"Georgian Bay Hotel, Trademark Collection by Wy..."
421,southbridge,US,42.0751,-72.0334,Comfort Inn & Suites


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))